# In This Exercise We Are Going To:
1. Wrangle some data in Spark regarding reviews and top-level categories
2. Write our DataFrame to a table we can access from Tableau 
3. Do some profiling in Tableau
4. Export our Tableau chart and embed it back into out notebook

The purpose of this notebook is to illustrate exporting embedding Tableau charts as image back into a Jupyter notebook.
For that reason, you will need to run the calculations, but the code for all of the wrangling and writing the table already exists in this notebook.  

Please start this exercise at the beginning of class (after spinning up a cluster and attaching this notebook) and work through until you reach the stop sign.

# Function for Embedding Images
In Databricks, you can load an image file to DBFS and embed it in your notebook, but only you and people in your organization (your
teammates that you invited) can see the image in the notebook.  If you export the notebook, the image does ***NOT*** come along for the 
ride, you only exported a link and it's to your workspace (sad).  However, there are (at least) three approaches you could use:

1. If you had a web server, you could load the exported images from Tableau on your server and make them publicly accessible
2. You could publish your notebook on Tableau Public and include a link
3. Embed the image as a base64 string and display it 

In this notebook, you will use the last approach, embedding a base64 encoded image.  The cell below 
defines a function named `showimage` that you will use in a code cell later in the notebook to embed your images from Tableau. The
function takes 1 required parameter and one optional parameter:
* **path** is the required parameter and is the path to your image in DBFS.  Assuming you uploaded your image through the GUI interface, your image is in `/FileStore/tables`.  In that case, your path should start with `/dbfs/FileStore/tables/` followed by your file's name.  If you are not sure of your file's name, use the Databricks file utilities to list the /FileStore/tables/ directory: `dbutils.fs.ls("/FileStore/tables")`.
* **width**: is the optional parameter and defaults to 0 if not included.  This is the percentage you want the image to be (where 50 means 50% and 200 means twice the original size).  The height will automatically adjust to be proportional to the original image.  This only changes the display, not the actual data file, so you can keep adjusting.  If you enter 0, the original size of the image is used.

**What the following function does:**
1. Using the path provided, the code opens the file for reading as a binary file (the "rb" parameter in the `open` function).  We open it as a binary file because otherwise Python will think we are opening a text file.
2. We read the file and pass that binary data to the b64encode function.  This function is reading each pixel of our image and turning it into a string.  Our image is turned into a very long (and humanly unreadable) string of characters.
3. We also open the file as an image.  The reason for this is that the Image class has methods that allow us to determine the width and height of the image (in pixels).
4. We then create some HTML (which like all HTML is just text that uses a specific encoding with tags), and this HTML contains a `div` tag with an `img` tag for our image inside of it.  The data for our image, instead of being at some location such as a URL is instead embedded in that image tag using the base64 string representation.  We use a bit of CSS to set the style for our `div` tag so it accommodates the size of our image.
5. If the width parameter was provided, the image is resized.
6. The HTML that's created is returned by the function.

In [0]:
from PIL import Image
TEMP_DIR = "/temp"

def getWidth(path):
  with Image.open(path) as img:
    width, height = img.size
    return(width)
  
def getDbfsPathName(path):
    # Get the fileinfo containing the path and name
  if path.startswith("/dbfs") != True:
    raise Exception("The path provided does not start with /dbfs")
  new_path = "dbfs:" + path[5:]
  # get the file info for the path
  file_list = dbutils.fs.ls(new_path)
  if len(file_list) != 1:
    raise Exception("The path provided is not a single file on dbfs")
  dbfs_path = file_list[0].path
  filename = file_list[0].name
  return(dbfs_path, filename)
  
def getTempPath(filename):
  # Create the temp directory if it does not exist
  temp_path = "file:" + TEMP_DIR
  dbutils.fs.mkdirs(temp_path)
  temp_list = dbutils.fs.ls(temp_path)
  # get a name to use for the copy
  temp_files = []
  for info in temp_list:
    temp_files.append(info.name)
  increment = 0
  new_name = filename
  while new_name in temp_files:
    increment+=1
    new_name = filename + "." + str(increment)
  access_path = TEMP_DIR + "/" + new_name # used for file opening
  return(access_path)

In [0]:
import base64
from PIL import Image

def showimage(path, width=0):
  image_string = ""
  img_tag = ""
  dbfs_path, filename = getDbfsPathName(path)
  access_path = getTempPath(filename)
  # copy the file
  copy_path = "file:" + access_path
  dbutils.fs.cp(dbfs_path,copy_path)
  with open(access_path, "rb") as image_file:
    image_string = base64.b64encode(image_file.read() ).decode('utf-8') 
    
  # Is the width setting a positive integer?  A width of 50 means 50%
  if width > 0 and width < 1:
    print("If the width parameter is specified, it must be 1 or more.  A width of 50 means 50%. The width entered was " + str(width) + ", so the original image width was used.")
    width = 0 #reset
    
  if width == 0:
    height = 0
    # Get the width and height of the image in pixels
    with Image.open(access_path) as img:
      width, height = img.size
      
    framewidth = width * 1.1
    # Build the image tag
    img_tag = '''
    <style>
    div {
      min-width: %ipx;
      max-width: %ipx;
    }
    </style>
    <div><img src="data:image/png;base64, %s"  style="width:%ipx;height=%ipx;" /></div>''' % (framewidth,framewidth,image_string, width, height)
  else: # a width was specified
    originalWidth = getWidth(access_path)
    imagewidth = int( width / 100.0 * originalWidth)
    framewidth = int( imagewidth * 1.1 )
    # Build the image tag
    img_tag = '''
    <style>
    div {
      min-width: %ipx;
      max-width: %ipx;
    }
    </style>
    <div><img src="data:image/png;base64, %s"  width="%ipx" height="auto"></div>''' % (framewidth,framewidth,image_string, imagewidth)
  # Clean up the file
  dbutils.fs.rm(copy_path)
  return(img_tag)

# Part 1: Profiling Data in Tableau

There is much profiling we can do in Jupyter, but Tableau provides a richer set of visualization capabilities.
In this exercise we will first use Spark to build our dataset and then export it to Tableau.  

First, we load the business data as `df_business1` in the following cell, print a record count, and print the schema.  If your file is on a 
different path, you will need to update the code below.

In [0]:
df_business1 = spark.read.json("/yelp/business.bz2")
print("business count:", df_business1.count() )
df_business1.printSchema()

business count: 209393
root
-- address: string (nullable = true)
-- attributes: struct (nullable = true)
 |-- AcceptsInsurance: string (nullable = true)
 |-- AgesAllowed: string (nullable = true)
 |-- Alcohol: string (nullable = true)
 |-- Ambience: string (nullable = true)
 |-- BYOB: string (nullable = true)
 |-- BYOBCorkage: string (nullable = true)
 |-- BestNights: string (nullable = true)
 |-- BikeParking: string (nullable = true)
 |-- BusinessAcceptsBitcoin: string (nullable = true)
 |-- BusinessAcceptsCreditCards: string (nullable = true)
 |-- BusinessParking: string (nullable = true)
 |-- ByAppointmentOnly: string (nullable = true)
 |-- Caters: string (nullable = true)
 |-- CoatCheck: string (nullable = true)
 |-- Corkage: string (nullable = true)
 |-- DietaryRestrictions: string (nullable = true)
 |-- DogsAllowed: string (nullable = true)
 |-- DriveThru: string (nullable = true)
 |-- GoodForDancing: string (nullable = true)
 |-- GoodForKids: string (nullable = true)
 |-- GoodForMeal: string (nullable = true)
 |-- HairSpecializesIn: string (nullable = true)
 |-- HappyHour: string (nullable = true)
 |-- HasTV: string (nullable = true)
 |-- Music: string (nullable = true)
 |-- NoiseLevel: string (nullable = true)
 |-- Open24Hours: string (nullable = true)
 |-- OutdoorSeating: string (nullable = true)
 |-- RestaurantsAttire: string (nullable = true)
 |-- RestaurantsCounterService: string (nullable = true)
 |-- RestaurantsDelivery: string (nullable = true)
 |-- RestaurantsGoodForGroups: string (nullable = true)
 |-- RestaurantsPriceRange2: string (nullable = true)
 |-- RestaurantsReservations: string (nullable = true)
 |-- RestaurantsTableService: string (nullable = true)
 |-- RestaurantsTakeOut: string (nullable = true)
 |-- Smoking: string (nullable = true)
 |-- WheelchairAccessible: string (nullable = true)
 |-- WiFi: string (nullable = true)
-- business_id: string (nullable = true)
-- categories: string (nullable = true)
-- city: string (nullable = true)
-- hours: struct (nullable = true)
 |-- Friday: string (nullable = true)
 |-- Monday: string (nullable = true)
 |-- Saturday: string (nullable = true)
 |-- Sunday: string (nullable = true)
 |-- Thursday: string (nullable = true)
 |-- Tuesday: string (nullable = true)
 |-- Wednesday: string (nullable = true)
-- is_open: long (nullable = true)
-- latitude: double (nullable = true)
-- longitude: double (nullable = true)
-- name: string (nullable = true)
-- postal_code: string (nullable = true)
-- review_count: long (nullable = true)
-- stars: double (nullable = true)
-- state: string (nullable = true)

### Selecting business fields
The question we want to ask is whether if we look at the review data, do the star ratings for some categories vary more widely than for others?

Possibly there are categories that Yelpers either love 'em or hate 'em (5 stars or 1 star), or possibly only review in those cases.

To answer these questions, we at least need the business ID and the categories they are in.  We will also include a few other fields.  Below we select the following fields from the `df_business1` DataFrame and name it `df_business2`:
* business_id
* name
* categories
* state  (but we are going to set "SC" to "NC")

Since this is the business data we will use throughout the rest of the notebook, we `cache` the DataFrame so Spark keeps it in memory.

We also print out the record count (it should match our original DataFrame) and show the first 5 records.  We make sure not to truncate the data or 
we wouldn't see all of the categories.

In [0]:
import pyspark.sql.functions as f

df_business2 = df_business1.select("business_id","name","categories",
                                   f.when( df_business1.state == 'SC', 'NC').otherwise( df_business1.state).alias("state")  ).cache()
print ("business count:", df_business2.count() )
df_business2.show(5, truncate=False)

business count: 209393
+----------------------+------------------------+--------------------------------------------------------------------------+-----+
business_id |name |categories |state|
+----------------------+------------------------+--------------------------------------------------------------------------+-----+
f9NumwFMBDn751xgFiRbNA|The Range At Lake Norman|Active Life, Gun/Rifle Ranges, Guns & Ammo, Shopping |NC |
Yzvjg0SayhoZgCljUJRF9Q|Carlos Santo, NMD |Health & Medical, Fitness & Instruction, Yoga, Active Life, Pilates |AZ |
XNoUzKckATkOD1hP6vghZg|Felinus |Pets, Pet Services, Pet Groomers |QC |
6OAZjbxqM5ol29BuHsil3w|Nevada House of Hose |Hardware Stores, Home Services, Building Supplies, Home & Garden, Shopping|NV |
51M2Kk903DFYI6gnB5I6SQ|USE MY GUY SERVICES LLC |Home Services, Plumbing, Electricians, Handyman, Contractors |AZ |
+----------------------+------------------------+--------------------------------------------------------------------------+-----+
only showing top 5 rows

### Flattening categories
The categories is a comma-separated list as a string (not a python list or array).  a couple examples (the 5 records you see could be different):

Minhas Micro Brewery: Tours, Breweries, Pizza, Restaurants, Food, Hotels & Travel

Geico Insurance: Insurance, Financial Services

We can flatten the categories:
* First split the string based on commas (and use the regular expression `\s*` to catch any spaces on either side of the comma).  Keep in mind that if you are doing this inside a Spark SQL query that you should use `\\\s*` instead of `\s*`.
* Then use the `EXPLODE` function to flatten out the array returned by the split. Explode will create separate lines for each element of the categories array, so if a business had 3 categories in its categories array, then after exploding there would be three lines in the resulting DataFrame for that business.  All of the other columns for that business' original line of data are duplicated for the three rows included in the result.    See the SQL Examples notebook for an example of using `EXPLODE` inside a Spark SQL query.

The record count (it should be 872,794).

In [0]:
import pyspark.sql.functions as f

df_business3 = df_business2.select("business_id","name","state",\
                                   f.explode(f.split(df_business2.categories, "\s*,\s*")).alias("category") ).cache()
print("record count:", df_business3.count() )
df_business3.show(truncate=False)

record count: 872794
+----------------------+------------------------+-----+---------------------+
business_id |name |state|category |
+----------------------+------------------------+-----+---------------------+
f9NumwFMBDn751xgFiRbNA|The Range At Lake Norman|NC |Active Life |
f9NumwFMBDn751xgFiRbNA|The Range At Lake Norman|NC |Gun/Rifle Ranges |
f9NumwFMBDn751xgFiRbNA|The Range At Lake Norman|NC |Guns & Ammo |
f9NumwFMBDn751xgFiRbNA|The Range At Lake Norman|NC |Shopping |
Yzvjg0SayhoZgCljUJRF9Q|Carlos Santo, NMD |AZ |Health & Medical |
Yzvjg0SayhoZgCljUJRF9Q|Carlos Santo, NMD |AZ |Fitness & Instruction|
Yzvjg0SayhoZgCljUJRF9Q|Carlos Santo, NMD |AZ |Yoga |
Yzvjg0SayhoZgCljUJRF9Q|Carlos Santo, NMD |AZ |Active Life |
Yzvjg0SayhoZgCljUJRF9Q|Carlos Santo, NMD |AZ |Pilates |
XNoUzKckATkOD1hP6vghZg|Felinus |QC |Pets |
XNoUzKckATkOD1hP6vghZg|Felinus |QC |Pet Services |
XNoUzKckATkOD1hP6vghZg|Felinus |QC |Pet Groomers |
6OAZjbxqM5ol29BuHsil3w|Nevada House of Hose |NV |Hardware Stores |
6OAZjbxqM5ol29BuHsil3w|Nevada House of Hose |NV |Home Services |
6OAZjbxqM5ol29BuHsil3w|Nevada House of Hose |NV |Building Supplies |
6OAZjbxqM5ol29BuHsil3w|Nevada House of Hose |NV |Home & Garden |
6OAZjbxqM5ol29BuHsil3w|Nevada House of Hose |NV |Shopping |
51M2Kk903DFYI6gnB5I6SQ|USE MY GUY SERVICES LLC |AZ |Home Services |
51M2Kk903DFYI6gnB5I6SQ|USE MY GUY SERVICES LLC |AZ |Plumbing |
51M2Kk903DFYI6gnB5I6SQ|USE MY GUY SERVICES LLC |AZ |Electricians |
+----------------------+------------------------+-----+---------------------+
only showing top 20 rows

# Identifying the top categories at each business

Instead of looking at every category, we are going to look at the ratings by top-level categories.  To do this, we first need to 
load the categories as you did in prior notebooks.  

In the next cell we are loading the data into a DataFrame named `df_categories`, printing the record count (there should be 1,565 categories), and 
printing the schema (that will make it easier to write queries against the DataFrame).  If your categories.json file is in a different path, 
you will need to update the code below.

In [0]:
df_categories = spark.read.option("multiline","true").json("/yelp/categories.json")
print( "number of categories:", df_categories.count() )
df_categories.show()
df_categories.printSchema()

number of categories: 1565
+------------------+--------------------+-----------------+---------------+-------------------+
 alias| country_blacklist|country_whitelist| parents| title|
+------------------+--------------------+-----------------+---------------+-------------------+
 3dprinting| null| null|[localservices]| 3D Printing|
 abruzzese| null| [IT]| [italian]| Abruzzese|
 absinthebars| null| [CZ]| [bars]| Absinthe Bars|
 acaibowls|[AR, TR, CL, PL, ...| null| [food]| Acai Bowls|
 accessories| null| null| [fashion]| Accessories|
 accountants| null| null| [professional]| Accountants|
 acnetreatment| null| null| [beautysvc]| Acne Treatment|
 active| null| null| []| Active Life|
 acupuncture| null| null| [health]| Acupuncture|
 addictionmedicine| [JP]| null| [physicians]| Addiction Medicine|
 adoptionservices| null| null|[localservices]| Adoption Services|
 adult| null| null| [shopping]| Adult|
 adultedu| null| null| [education]| Adult Education|
adultentertainment| null| null| [nightlife]|Adult Entertainment|
 advertising| null| null| [professional]| Advertising|
 aerialfitness| null| null| [fitness]| Aerial Fitness|
 aerialtours| null| null| [tours]| Aerial Tours|
 aestheticians|[AR, CH, CL, BE, ...| null| [medicalspa]| Aestheticians|
 afghani| [MX, TR]| null| [restaurants]| Afghan|
 african| [TR]| null| [restaurants]| African|
+------------------+--------------------+-----------------+---------------+-------------------+
only showing top 20 rows

root
-- alias: string (nullable = true)
-- country_blacklist: array (nullable = true)
 |-- element: string (containsNull = true)
-- country_whitelist: array (nullable = true)
 |-- element: string (containsNull = true)
-- parents: array (nullable = true)
 |-- element: string (containsNull = true)
-- title: string (nullable = true)

### Identifying the top-level categories
All we really need is the top-level categories, which from a prior exercise we know these are the categories with 
an empty array for the parents.  Since all we need is the `title` column to match with the user data, in the next cell
we create a new DataFrame named `df_topCategories` that has just the `title` column for the top categories.  We show all of the 
categories in the new DataFrame (there should be 22 rows).

In [0]:
df_topCategories = df_categories.filter(f.size(df_categories.parents) == 0).select("title").cache()
df_topCategories.show(30,truncate=False)

+----------------------------+
title |
+----------------------------+
Active Life |
Arts & Entertainment |
Automotive |
Beauty & Spas |
Bicycles |
Education |
Event Planning & Services |
Financial Services |
Food |
Health & Medical |
Home Services |
Hotels & Travel |
Local Flavor |
Local Services |
Mass Media |
Nightlife |
Pets |
Professional Services |
Public Services & Government|
Religious Organizations |
Restaurants |
Shopping |
+----------------------------+

### Top Categories for each business
We need to do an INNER JOIN of the business data in the DataFrame `df_business3` and the top-level categories
in `df_topCategories`.  We want to join where the `category` field in the business data matches the `title`
field in the top level business data.  We just need to include the fields from the business data in the result.

You can do the join either using PySpark or the `sql` method of the SparkSession to run a SQL query, but 
just remember you first need to create temporary views for the `df_business3` and `df_topCategories` 
DataFrames using the `createOrReplaceTempView` method.

We name the new DataFrame `df_busCategories` and print the record count (there should be 295,769 records).

In [0]:
df_business3.createOrReplaceTempView("business")
df_topCategories.createOrReplaceTempView("categories")

In [0]:
df_busCategories = spark.sql("""
SELECT B.*
FROM business AS B INNER JOIN categories AS C
ON B.category = C.title
""").cache()
print("record count:", df_busCategories.count() )
df_busCategories.show(truncate=False)
df_business3.unpersist()
df_topCategories.unpersist()

record count: 295769
+----------------------+------------------------------+-----+----------------+
business_id |name |state|category |
+----------------------+------------------------------+-----+----------------+
f9NumwFMBDn751xgFiRbNA|The Range At Lake Norman |NC |Active Life |
f9NumwFMBDn751xgFiRbNA|The Range At Lake Norman |NC |Shopping |
Yzvjg0SayhoZgCljUJRF9Q|Carlos Santo, NMD |AZ |Health & Medical|
Yzvjg0SayhoZgCljUJRF9Q|Carlos Santo, NMD |AZ |Active Life |
XNoUzKckATkOD1hP6vghZg|Felinus |QC |Pets |
6OAZjbxqM5ol29BuHsil3w|Nevada House of Hose |NV |Home Services |
6OAZjbxqM5ol29BuHsil3w|Nevada House of Hose |NV |Shopping |
51M2Kk903DFYI6gnB5I6SQ|USE MY GUY SERVICES LLC |AZ |Home Services |
cKyLV5oWZJ2NudWgqs8VZw|Oasis Auto Center - Gilbert |AZ |Automotive |
oiAlXZPIFm2nBCt0DHLu_Q|Green World Cleaners |NV |Local Services |
ScYkbYNkDgCneBrD9vqhCQ|Junction Tire & Auto Service |AZ |Automotive |
pQeaRpvuhoEqudo3uymHIQ|The Empanadas House |IL |Food |
pQeaRpvuhoEqudo3uymHIQ|The Empanadas House |IL |Restaurants |
EosRKXIGeSWFYWwpkbhNnA|Xtreme Couture |ON |Active Life |
MbZMmwo-eL0Jnm_Yb9KJrA|Chinook Landscaping and Design|AB |Home Services |
7Dv4_HAxsxvadEsT5fxQBg|Dependable Brakes & Exhaust |PA |Automotive |
M_guz7Dj7hX0evS672wIwA|Chocolate Shoppe Ice Cream |WI |Food |
JjJs3o60uQCfctDjs45cmA|Convertabath |AZ |Home Services |
JjJs3o60uQCfctDjs45cmA|Convertabath |AZ |Local Services |
kOICO53wbOiOJcKuCgOQ3A|Tan Las Vegas |NV |Beauty & Spas |
+----------------------+------------------------------+-----+----------------+
only showing top 20 rows

Out[31]: DataFrame[title: string]

### Weighting the categories
As you can see in just the small snapshot you printed above, although some businesses are in a single category, 
many businesses are in multiple top-level categories.  If we match up these business records with the 
reviews, some businesses could be seen as having a disproportionate effect since their reviews count in multiple 
categories.  Possibly that's OK, but for this exercise we are going to weight them, so if a business is in 3 categories, 
we will weight each review as 1/3 for each category.

To do this, we are first going to count how many top-level categories each business is in.  we will then join that result back with the 
data we have in `df_busCategories` to assign a weight to each record that is 1 divided by the number of top-level categories the 
business is in.

In the next cell, we group the business by the `business_id` and get a count.  You can use either PySpark or the `sql` method from the SparkSession.
In either case, we call the new DataFrame `df_busCount` and the field with the number of categories for each business is named `catCount`.  
You would expect to end up with 209,393 records (the same number of records that were in the business data you loaded); but you don't. **WHY?**

In [0]:
df_busCount = df_busCategories.groupBy("business_id").count().toDF("business_id", "catCount")
print("record count:", df_busCount.count() )
df_busCount.show(truncate=False)

record count: 208869
+----------------------+--------+
business_id |catCount|
+----------------------+--------+
M_guz7Dj7hX0evS672wIwA|1 |
oqbNgOBAxsa49c2WQWHL-w|2 |
2BKKUjLSocdlgfsMC2RSGQ|1 |
DloBjCZHlz0lzsrQJq5PHQ|3 |
lj0MiK5_fyv9df2twnsI7g|1 |
WtjgLPHTCYtqiZ31KBRDHQ|1 |
x1fx7C_tc064cFATjl4fmQ|3 |
s5345m9QBKwZrliFQ9FcfA|2 |
RasF3XQRsFqfmY596ixKhA|1 |
ATeOjZzE3VOI-RW7FUFJpQ|2 |
VHsNB3pdGVcRgs6C3jt6Zg|1 |
AucmLoCNGDkdg8sn6nxv1Q|1 |
RMjCnixEY5i12CiqnskiVw|1 |
BQMVcJQV70gUQjwi7-MrSg|1 |
M9N2eaO8PkT7MHGLLA6V3Q|1 |
wwl11NPZg4KehKNLx28_nw|2 |
YYztMMdcudL4xuSF29yelQ|1 |
aZbbt_RPKV8vUY7GWiEEyw|1 |
KqRn57aegoaDmbHqbqRwKw|2 |
ei7SYxd07uETpASn1KyY_g|1 |
+----------------------+--------+
only showing top 20 rows

### What happened to the missing records?
We started with 209,393 records; one for each business.  We then exploded their categories and summed them back up by 
`business_id`, so it would seem we should have the same number of records.  To check why we lost businesses in the process, 
in the following cell we do a LEFT ANTI JOIN (see the Spark SQL reading for details for a description of this join).
This returns all of the records in our original data, but not in the `df_busCount` DataFrame.  See the pattern?

Hint: look at the `categories` column

In [0]:
df_missing = df_business2.join(df_busCount,df_business2.business_id == df_busCount.business_id,"left_anti")
print("records:", df_missing.count() )
df_missing.show(truncate=False)

records: 524
+----------------------+-----------------------------------------+----------+-----+
business_id |name |categories|state|
+----------------------+-----------------------------------------+----------+-----+
2W1tLg8ybRUEKMPoAPHTsQ|Cox Media |null |NV |
oCbk_T5eyc3KB0iT6T5q9w|Criminal Customz |null |OH |
4YI_gJQuW-8d_o2xvflJxA|Cag Acceptance |null |AZ |
reOlnE49Ug3BMSGn3IAweg|Csorba Tree Experts |null |PA |
KnAV3ZhOWWNhqnvqIkxYOg|Accurate Lien & Contractor Assistance |null |AZ |
pG8D1gthGbMMOj9y1MxOeA|Cub Cadet |null |OH |
3stT022u8m1H37mcopGuUA|Flik & Company |null |ON |
HgJqVPLNI29YJIlYBmScQQ|Ice Man of Arizona Equipment Company |null |AZ |
_lVdg7Tdw-Vkn0dn-gS2qw|Vascular Access Center of Southern Nevada|null |NV |
Fjd69v08ozCbV2qEbWkLCQ|Grant and Weber |null |NV |
oYFUHj8XFM36SdPCoQaXPQ|Austin's Automotive Inspection Service |null |NC |
8eHUD8OBe2nXaLKV68K0Rw|Fortune Properties |null |NV |
_idu2VZT3e2xy1tGx9P4fA|Greenshield Canada |null |ON |
5qv5MOZh1UTmj18-5zSusQ|Trugreen Chemlawn |null |NC |
IBAvWgl1CX8doybLdA-74g|Peoples Utility |null |NV |
pm36bCxGC_hPkLMhiObBDw|Arizona Pipeline Company |null |AZ |
hEQxSi_Mo5xXtSFnK9trSw|City Tin Shop |null |NC |
9dAHyMaKyE1Rzl9U1AKaBQ|Davita #3682 |null |NV |
rWsoiLQeFy-RCdYtQvH-JA|Budget Rent A Car System |null |NV |
BiBTwivxbsVVlNVgj4lduA|M & D Limited |null |OH |
+----------------------+-----------------------------------------+----------+-----+
only showing top 20 rows

### Profiling to confirm our hunch
In the following cell, write a query against the `df_business2` DataFrame to confirm our hunch regarding the 524 businesses that were not included
in the `df_busCount` DataFrame.

In [0]:
df_business2.filter("categories IS NULL").count()

Out[34]: 524

#### Cleaning up
We are now done with the `df_business2` DataFrame, so we can unpersist it.

In [0]:
df_business2.unpersist()

Out[35]: DataFrame[business_id: string, name: string, categories: string, state: string]

### Setting the category weight

In [0]:
df_busCategories.createOrReplaceTempView("busCat")
df_busCount.createOrReplaceTempView("busCount")

df_weighted = spark.sql("""
SELECT B.*, (1 / C.catCount) AS weight
FROM busCat AS B INNER JOIN busCount AS C
ON B.business_id = C.business_id
""").cache()
print("record count:", df_weighted.count() )
df_weighted.show(truncate=False)
df_busCategories.unpersist()

record count: 295769
+----------------------+----------------------------------------+-----+---------------------+------+
business_id |name |state|category |weight|
+----------------------+----------------------------------------+-----+---------------------+------+
--9e1ONYQuAa-CB_Rrw7Tw|Delmonico Steakhouse |NV |Restaurants |1.0 |
-6c_bJblLXUwoWfmYLtH_w|Neo-Paws International |ON |Pets |1.0 |
-ElDqujEn1u64ynQr0bX7g|McFarling Law Group |NV |Professional Services|1.0 |
-Gh9a15ijNii-8rnp4x3xA|Volvo of Mississauga |ON |Automotive |1.0 |
-I06hkMFrX0KBqu61ik1YA|Andre Agassi College Preparatory Academy|NV |Education |1.0 |
-Qb6U1MotJfpt6OU_uvqQA|Woof Wise Dog Training |OH |Pets |1.0 |
-RRnldwSqCZT6OpuWKXolg|Grandiose |AZ |Shopping |1.0 |
-Spbg8zeDHA49Ys_f8WEIw|The Elf Squad |NC |Shopping |1.0 |
-TGwtTiieh_JLml5_VRMQA|Starbucks |ON |Food |1.0 |
-VAsjhmAbKF3Pb_-8rh3xg|McDonald's |QC |Restaurants |0.5 |
-VAsjhmAbKF3Pb_-8rh3xg|McDonald's |QC |Food |0.5 |
-WGjtt88-6zBiIUiKhJjgg|The Nail Salon & Spa |ON |Beauty & Spas |1.0 |
-cxD1NimFldATDUsN-oa3A|Coasta Cantina |NV |Restaurants |1.0 |
-m8v19CXrGEYr3PALjqVCg|Daystar Boutique |OH |Shopping |1.0 |
-qL-wX-UuMaHDBA2X5G4kw|The Thumb |AZ |Automotive |1.0 |
-r8SvItXXG6_T3mP5GXRAw|Western Coffee Shop |AB |Restaurants |0.5 |
-r8SvItXXG6_T3mP5GXRAw|Western Coffee Shop |AB |Food |0.5 |
-tUFV7Zk3RCvPL8QIDG9Bg|Pure Healthy Cosmestic Spa |NV |Health & Medical |0.5 |
-tUFV7Zk3RCvPL8QIDG9Bg|Pure Healthy Cosmestic Spa |NV |Beauty & Spas |0.5 |
-wby-wyqaq1ZDTFDTdKxQA|Trendy Wall Designs |NV |Shopping |0.25 |
+----------------------+----------------------------------------+-----+---------------------+------+
only showing top 20 rows

Out[36]: DataFrame[business_id: string, name: string, state: string, category: string]

# Matching Businesses with Review Data

We now want to match our business and review data to associate each review with the categories for that business.

In the following cell we load the reviews as `df_reviewData`.

In your project, you should generally have a cell that just does the import so you can see the whole schema.  Here we are going to also select only the following fields:

* review ID
* business ID
* stars

In the following cell, we create a DataFrame named `df_reviews` that selects the above listed columns (the actual column names are listed in the schema).

Since we have included only the columns we need, we will `cache` the DataFrame.

In [0]:
if ( spark.catalog._jcatalog.tableExists("reviews_without_text_table") ):
  print("Loading review data from table ...")
  df_reviewData = spark.read.format('parquet').table("reviews_without_text_table").cache()
  print("loaded review data from table")
else:
  print("The Building Review and User Tables notebook has not been run, loading review data from files ...")
  df_reviewData = spark.read.json("/yelp/review.bz2").select("business_id","user_id","cool","funny","useful","date","stars").cache()
  # Does not already exist as a table, so write it out
  df_reviewData.write.mode("overwrite").format('parquet').saveAsTable("reviews_without_text_table")
  print("loaded review data from file")

df_reviews = df_reviewData.select("business_id", "stars")
df_reviewData.unpersist()
print("review count:", df_reviews.count() )
df_reviews.show(truncate=False)

Loading review data from table ...
loaded review data from table
review count: 8021122
+----------------------+-----+
business_id |stars|
+----------------------+-----+
3chd-CjwQz7Pap_MQEWhWg|4.0 |
XV45a3fz1d7SRoash2vu3g|3.0 |
BxcLVgA6BndKDJol0ZZu0g|3.0 |
nSPh4_BKX3Fch0TOahe-vQ|1.0 |
KrcmoxVBX9o3OxFogGx-RA|3.0 |
ewn93D_Q6bnVosJtwF9oiQ|2.0 |
fKjeOlJTQ1W0cSDRYFdZmA|1.0 |
b7xistZfcbSO6_0Mg-heRA|1.0 |
6pM2YxQH8dQYs07_CH3Z6g|5.0 |
iVjeR9z1xVt2ZpxVU4UcXQ|5.0 |
t0edCFMUYmFSN_eXT4HrnQ|4.0 |
TumOl-iIDkZFRM1k95USAg|5.0 |
ndjiWXoH1pPMwEBhRTnitA|5.0 |
z8lagvcw1Y4Pz4dVNRO48g|5.0 |
0OZckGHdpbhMMy8ex2MRBQ|5.0 |
d10IxZPirVJlOSpdRZJczA|1.0 |
J3EW9_IQQqkp4nN3q0Mcaw|4.0 |
E9HkUYYO_-5vKjvBwLuDIw|5.0 |
lNBhwf_3c0QBqBlsYRygaQ|4.0 |
Kp4i7on7rxFRkWceKfL2Ig|5.0 |
+----------------------+-----+
only showing top 20 rows

### Join the Reviews and Business data based on business ID
We want to bring the review and business data together, so we will do an INNER JOIN
between the reviews and business data.  We want most of the fields in both 
tables, but we only want the business ID once.

In [0]:
df_busReviews = df_reviews.join(df_weighted, df_reviews.business_id == df_weighted.business_id).\
select(df_reviews.business_id, "stars", "name", "state", "category", "weight").cache()
print("record count:", df_busReviews.count() )
df_busReviews.show(truncate=False)
df_reviews.unpersist()

record count: 13029836
+----------------------+-----+--------------------+-----+-----------+------+
business_id |stars|name |state|category |weight|
+----------------------+-----+--------------------+-----+-----------+------+
--9e1ONYQuAa-CB_Rrw7Tw|5.0 |Delmonico Steakhouse|NV |Restaurants|1.0 |
--9e1ONYQuAa-CB_Rrw7Tw|5.0 |Delmonico Steakhouse|NV |Restaurants|1.0 |
--9e1ONYQuAa-CB_Rrw7Tw|5.0 |Delmonico Steakhouse|NV |Restaurants|1.0 |
--9e1ONYQuAa-CB_Rrw7Tw|2.0 |Delmonico Steakhouse|NV |Restaurants|1.0 |
--9e1ONYQuAa-CB_Rrw7Tw|2.0 |Delmonico Steakhouse|NV |Restaurants|1.0 |
--9e1ONYQuAa-CB_Rrw7Tw|5.0 |Delmonico Steakhouse|NV |Restaurants|1.0 |
--9e1ONYQuAa-CB_Rrw7Tw|5.0 |Delmonico Steakhouse|NV |Restaurants|1.0 |
--9e1ONYQuAa-CB_Rrw7Tw|5.0 |Delmonico Steakhouse|NV |Restaurants|1.0 |
--9e1ONYQuAa-CB_Rrw7Tw|5.0 |Delmonico Steakhouse|NV |Restaurants|1.0 |
--9e1ONYQuAa-CB_Rrw7Tw|4.0 |Delmonico Steakhouse|NV |Restaurants|1.0 |
--9e1ONYQuAa-CB_Rrw7Tw|1.0 |Delmonico Steakhouse|NV |Restaurants|1.0 |
--9e1ONYQuAa-CB_Rrw7Tw|2.0 |Delmonico Steakhouse|NV |Restaurants|1.0 |
--9e1ONYQuAa-CB_Rrw7Tw|4.0 |Delmonico Steakhouse|NV |Restaurants|1.0 |
--9e1ONYQuAa-CB_Rrw7Tw|5.0 |Delmonico Steakhouse|NV |Restaurants|1.0 |
--9e1ONYQuAa-CB_Rrw7Tw|5.0 |Delmonico Steakhouse|NV |Restaurants|1.0 |
--9e1ONYQuAa-CB_Rrw7Tw|3.0 |Delmonico Steakhouse|NV |Restaurants|1.0 |
--9e1ONYQuAa-CB_Rrw7Tw|5.0 |Delmonico Steakhouse|NV |Restaurants|1.0 |
--9e1ONYQuAa-CB_Rrw7Tw|5.0 |Delmonico Steakhouse|NV |Restaurants|1.0 |
--9e1ONYQuAa-CB_Rrw7Tw|5.0 |Delmonico Steakhouse|NV |Restaurants|1.0 |
--9e1ONYQuAa-CB_Rrw7Tw|1.0 |Delmonico Steakhouse|NV |Restaurants|1.0 |
+----------------------+-----+--------------------+-----+-----------+------+
only showing top 20 rows

Out[38]: DataFrame[business_id: string, stars: double]

# Write the data to a table we can access in Tableau
Since we are going to be using Tableau Desktop, we are going to write the data to a table which we will access from Tableau via the ODBC dconnector we installed.

In [0]:
df_busReviews.write.mode("overwrite").saveAsTable("business_reviews")

# Visualizing in Spark

Before visualizing our data in Tableau, we will  visualize it in Spark.  
In the following cell we are going to visualize the percentage for each of the 
top categories based on the weighted reviews for each of the metro areas in our data.

In [0]:
display(df_busReviews.select("state","category", "weight").withColumnRenamed("weight","weighted_reviews"))

state category weighted_reviews NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1

### Cleaning up the visualization

We have two issues:
* Dirty data (we only want the states that are a proxy for the 10 metro areas)
* Limitations of Spark visualizations - we can only see twenty categories out of 22, so we will eliminate:
  * Bicycles (it should not apply in the U.S. or Canada anyways)
  * Relgion (it's a small category - not much data)

In [0]:
df_filteredReviews = df_busReviews.select("state","category", "weight").withColumnRenamed("weight","weighted_reviews").\
filter(df_busReviews.state.isin(['AB','AZ','IL','NC','NV','OH','ON','PA','QC','WI']) ).\
filter("category != 'Bicycles'").filter("category != 'Religious Organizations'")

In [0]:
display(df_filteredReviews)

state category weighted_reviews NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1.0 NV Restaurants 1

### Release the Data Being Held in Memory

The following cell is commented out.  When you have completed all of the work above and are moving to Tableau, uncomment this cell and run it.

In [0]:
# df_busReviews.unpersist()

# Part 2: We Are Going Away Now.... But As Arnold Said, "I'LL BE BACK!"
Before running the following cell, you need to have:
1. Done your profiling in Tableau
2. Exported the result as an image file (png)
3. Uploaded that file to DBFS
4. Copied the file path for your uploaded png file

### And now back to our previously scheduled program ...

### Distribution of Star Ratings Across Categories
To embed the image file you uploaded, put the file path for your image file in DBFS into the following cell.  When you uploaded your image,
it went into FileStore / tables by default (the web-accessible directory for uploads and downloads).  In the following cell, the path for FileStore / tables is:

`/dbfs/FileStore/tables/`

If you put your file in a subdirectory when uploading, that and the file name need to be appended to the above path.

The following cell calls a Databicks function named `displayHTML` which does exactly as its name implies.  The parameter you are passing to that function is a call to the 
`showimage` function at the start of this notebook.  That function generates some HTML that has your image embedded in it using base64 encoding.

If you are embedding images in the notebook for your team project, you will need to:
1. Use the `displayHTML` and `showimage` functions, which means you must also include the definition of `showimage` and the supporting functions.
2. Include a markdown  cell above the cell where you embed your image and that cell should say:
  * What processing was done in Tableau (not every step, but major steps where you filtered, calculated, or aggregated, and what's shown on each axis).
  * How you are interpreting the results shown in the image.  Interpretation does NOT mean, "bar 1 is the longest" (the reader should see that if it's true).  Instead your team should say how you interpret the result shown in the chart.
  
#### Resizing the cell with your image

The `showimage` function as defined sizes the image in the cell based on the size in the png file, but it makes the cell longer than you would want it to be.  Be sure to drag the corner of 
the cell up to resize the cell so there is not excessive blank space after the image.

In [0]:
displayHTML( showimage("/dbfs/FileStore/tables/Category_stars.png", 50) )